In [ ]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from mls_scf_tools.mls_pint import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]

In [ ]:
from pathlib import Path
import shutil
import warnings

In [ ]:
%autoreload 2

In [ ]:
import njl_corf.wrc27_views as wrc27_views

In [ ]:
ai_info = wrc27_views.get_ai_info()

In [ ]:
print(ai_info.keys())

In [ ]:
root = Path(r"/Users/livesey/corf/wrc-27-views-google-drive/")
python_root = Path(r"/Users/livesey/corf/njl-corf/sandbox/wrc27-views-sandbox/outputs/")
for title in ai_info.keys():
    print("=" * 30 + " " + title)
    wrc, ai = title.split(" ")
    # Only consider the "a" cases
    if not ai.endswith("a"):
        continue
    ai = ai[:-1]
    # Create the new directory if needed
    path = root / wrc / ai
    path.mkdir(parents=True, exist_ok=True)
    # Copy the relevant resolution into it
    wildcard = "*ai" + "-".join(ai[2:].split(".")) + ".pdf"
    resolution_candidates = list((root / "initial-triage").glob(wildcard))
    if not resolution_candidates:
        raise ValueError(f"Unable to find match for {wildcard}")
    if len(resolution_candidates) > 1:
        print(resolution_candidates)
        raise ValueError("Too many matches for {wildcard}.")
    resolution = Path(resolution_candidates[0])
    destination = path / resolution.name
    print(f"Destination is {destination}")
    if not destination.exists():
        shutil.copy(resolution, destination)
        pass
    else:
        print(f"Skipping {resolution}, file exists.")
    # Copy the html file(s) and figures into a data subdirectory therein
    data_path = path / "data"
    data_path.mkdir(exist_ok=True)
    wildcard = f"{wrc}_AI{ai[2:]}[abcdefg]*.*"
    files = list(python_root.glob(wildcard))
    for file in files:
        filename = Path(file).name
        destination = data_path / filename
        if not destination.exists():
            shutil.copy(file, destination)
            pass
        else:
            print(f"Skipping {filename}, file exists.")
    # Copy the template document to that directory
    template = root / "resources" / "AI-Template.docx"
    filename = wrc + "_" + "-".join(ai.split(".")) + "_views.docx"
    destination = path / filename
    if not destination.exists():
        # shutil.copy(template, destination)
        warnings.warn(
            f"Skipped copying to {destination}, no updates to documents allowed."
        )
        pass
    else:
        print(f"Skipping {filename}, file exists.")